In [1]:
## WeightWatcher helps you choose the best pretrained model for your needs.

## You can use WeightWatcher to compare several pretrained models and choose the one with the lowest Log Norm.
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = 8,8

from scipy.ndimage.interpolation import zoom
import numpy as np
import os
from PIL import Image
import weightwatcher as ww

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

# RoBERTa large

In [2]:
import torch
import torch.nn as nn
import transformers
from transformers import AutoModel, BertModel
import numpy as np

class RobertaLarge(nn.Module):
    def __init__(self):
        super(RobertaLarge, self).__init__()
        self.roberta = AutoModel.from_pretrained("roberta-large", output_hidden_states=True)
        self.drop_out = nn.Dropout(0.1) 
        self.l0 =  nn.Linear(1024 * 2, 1)
        torch.nn.init.normal_(self.l0.weight, std=0.02)

    def forward(self,ids,attention_mask):
        _, _, out = self.roberta(
            ids,
            attention_mask=attention_mask
        )
        out = torch.cat((out[-1], out[-2]), dim=-1)
        #out = self.drop_out(out)
        out = out[:,0,:]
        logits = self.l0(out)
        
        return logits

device = torch.device("cpu")
robertaLargeModel = RobertaLarge()
robertaLargeModel.to(device)
robertaLargeModel.load_state_dict(torch.load("/home/mkarim/BERTs/COVID19-Fake-News-Detection-in-English-submission_ready_branch/roberta-large/store_model/roerta_large_four_train_16_3e5.bin", map_location=torch.device('cpu')))

/usr/local/lib/python3.7/dist-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://s3.amazonaws.com:443 "HEAD /models.huggingface.co/bert/roberta-large-config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): cdn.huggingface.co:443
DEBUG:urllib3.connectionpool:https://cdn.huggingface.co:443 "HEAD /roberta-large-pytorch_model.bin HTTP/1.1" 200 0


<All keys matched successfully>

In [ ]:
robertaLargeModelwatcher = ww.WeightWatcher(model=robertaLargeModel)
robertaLargeModelDetails = robertaLargeModelwatcher.analyze()
robertaLargeModelSummary = robertaLargeModelwatcher.get_summary(robertaLargeModelDetails)

In [7]:
robertaLargeModelSummary

{'log_norm': 3.3858589208732903,
 'alpha': 4.305526190763662,
 'alpha_weighted': 7.556126318340793,
 'log_alpha_norm': 7.8375270141688365,
 'log_spectral_norm': 1.8241622140546523,
 'stable_rank': 47.87746939962413}

## Twitter berts